<a href="https://colab.research.google.com/github/SilvanaBContreras/AA_TP1_G8/blob/main/AA_TP1_G8_clasif_binaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Levanto datos procesados en notebook EDA

In [2]:
URL_BASE = 'https://github.com/SilvanaBContreras/AA_TP1_G8/raw/refs/heads/main/'

archivos_a_cargar = {
    'X_train_completo.parquet': 'X_train',
    'y_train.parquet': 'y_train',
    'X_test_completo.parquet': 'X_test',
    'y_test.parquet': 'y_test',
    'X_train_reducido.parquet': 'X_train_short',
    'X_test_reducido.parquet': 'X_test_short',
}

def cargar_conjunto(nombre_archivo):
    """Carga un archivo Parquet desde la URL de GitHub."""
    full_url = URL_BASE + nombre_archivo
    data = pd.read_parquet(full_url)

    # Si es un DataFrame con una sola columna (que era originalmente una Series 'y'),
    # la convertimos de vuelta a Series con .squeeze()
    if data.shape[1] == 1:
        return data.squeeze()

    return data

# Crear un diccionario para almacenar todas las variables cargadas
variables_cargadas = {}

# Cargar y asignar dinámicamente
for nombre_archivo, nombre_variable in archivos_a_cargar.items():
    datos = cargar_conjunto(nombre_archivo)
    variables_cargadas[nombre_variable] = datos
    print(f"✅ Cargado {nombre_archivo} como la variable '{nombre_variable}'")

# Asignar las variables al espacio de nombres de la notebook
X_train = variables_cargadas['X_train']
y_train = variables_cargadas['y_train']
X_test = variables_cargadas['X_test']
y_test = variables_cargadas['y_test']
X_train_short = variables_cargadas['X_train_short']
X_test_short = variables_cargadas['X_test_short']

print("\n--- Verificación Final ---")
print(f"Variable X_train cargada. Dimensiones: {X_train.shape}")
print(f"Variable X_train_short cargada. Dimensiones: {X_train_short.shape}")
print("¡Estás lista para comenzar a entrenar y comparar modelos! 🎉")

✅ Cargado X_train_completo.parquet como la variable 'X_train'
✅ Cargado y_train.parquet como la variable 'y_train'
✅ Cargado X_test_completo.parquet como la variable 'X_test'
✅ Cargado y_test.parquet como la variable 'y_test'
✅ Cargado X_train_reducido.parquet como la variable 'X_train_short'
✅ Cargado X_test_reducido.parquet como la variable 'X_test_short'

--- Verificación Final ---
Variable X_train cargada. Dimensiones: (40257, 51)
Variable X_train_short cargada. Dimensiones: (40257, 49)
¡Estás lista para comenzar a entrenar y comparar modelos! 🎉


In [3]:
print("Dimensiones del conjunto de características (X):")
print(f"X_train: {X_train.shape}")
print("\nDimensiones del conjunto de características (X) sin correlacionadas:")
print(f"X_train_short: {X_train_short.shape}")
print("\nDimensiones de la variable objetivo (y):")
print(f"X_test: {X_test.shape}")
print("\nDimensiones de la variable objetivo (y) sin correlaciones:")
print(f"X_test_short: {X_test_short.shape}")
print("\nDimensiones de la variable objetivo (y):")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

Dimensiones del conjunto de características (X):
X_train: (40257, 51)

Dimensiones del conjunto de características (X) sin correlacionadas:
X_train_short: (40257, 49)

Dimensiones de la variable objetivo (y):
X_test: (17253, 51)

Dimensiones de la variable objetivo (y) sin correlaciones:
X_test_short: (17253, 49)

Dimensiones de la variable objetivo (y):
y_train: (40257,)
y_test: (17253,)


# Regresión logística con correl

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, \
                            confusion_matrix, roc_auc_score

model = LogisticRegression(penalty=None, max_iter=5000)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Predicción de probabilidades para la clase positiva (necesaria para AUC-ROC)
y_pred_proba = model.predict_proba(X_test)[:, 1]


accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Matriz de Confusión (usando y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Métrica de Probabilidad (usando y_pred_proba)
auc_roc = roc_auc_score(y_test, y_pred_proba)

print("--- Evaluación del Modelo ---")
print(f"Accuracy de la clasificación: {accuracy:.4f}")
print(f"Recall de la clasificación: {recall:.4f}")
print(f"Precisión de la clasificación: {precision:.4f}")
print(f"F1-score de la clasificación: {f1:.4f}")
print(f"AUC-ROC de la clasificación: {auc_roc:.4f} \n(Métrica clave para desbalance)")

print("\n--- Matriz de Confusión ---")
print("La matriz se interpreta como: \n[[TN, FP], \n[FN, TP]]")
print(conf_matrix)


--- Evaluación del Modelo ---
Accuracy de la clasificación: 0.8560
Recall de la clasificación: 0.9163
Precisión de la clasificación: 0.8607
F1-score de la clasificación: 0.8876
AUC-ROC de la clasificación: 0.9167 
(Métrica clave para desbalance)

--- Matriz de Confusión ---
[[4959 1588]
 [ 896 9810]]


In [10]:
print("\n--- Matriz de Confusión ---")
print("La matriz se interpreta como: \n[[TN, FP], \n[FN, TP]]")
print(conf_matrix)


--- Matriz de Confusión ---
La matriz se interpreta como: 
[[TN, FP], 
[FN, TP]]
[[4959 1588]
 [ 896 9810]]


# Regresión logística SIN correl

In [8]:

model_short = LogisticRegression(penalty=None, max_iter=5000)

model_short.fit(X_train_short, y_train)

y_pred_short = model_short.predict(X_test_short)

# Predicción de probabilidades para la clase positiva (necesaria para AUC-ROC)
y_pred_proba_short = model_short.predict_proba(X_test_short)[:, 1]


accuracy_short = accuracy_score(y_test, y_pred_short)
recall_short = recall_score(y_test, y_pred_short)
precision_short = precision_score(y_test, y_pred_short)
f1_short = f1_score(y_test, y_pred_short)

# Matriz de Confusión (usando y_pred)
conf_matrix_short = confusion_matrix(y_test, y_pred_short)

# Métrica de Probabilidad (usando y_pred_proba)
auc_roc_short = roc_auc_score(y_test, y_pred_proba_short)

print("--- Evaluación del Modelo ---")
print(f"Accuracy_short de la clasificación: {accuracy_short:.4f}")
print(f"Recall_short de la clasificación: {recall_short:.4f}")
print(f"Precisión_short de la clasificación: {precision_short:.4f}")
print(f"F1-score _short de la clasificación: {f1_short:.4f}")
print(f"AUC-ROC _short de la clasificación: {auc_roc_short:.4f} \n(Métrica clave para desbalance)")

print("\n--- Matriz de Confusión  _short---")
print("La matriz se interpreta como: [[TN, FP], \n[FN, TP]]")
print(conf_matrix_short)

--- Evaluación del Modelo ---
Accuracy_short de la clasificación: 0.8428
Recall_short de la clasificación: 0.9063
Precisión_short de la clasificación: 0.8502
F1-score _short de la clasificación: 0.8774
AUC-ROC _short de la clasificación: 0.9066 
(Métrica clave para desbalance)

--- Matriz de Confusión  _short---
La matriz se interpreta como: [[TN, FP], 
[FN, TP]]
[[4838 1709]
 [1003 9703]]


In [9]:
print("\n--- Matriz de Confusión  _short---")
print("La matriz se interpreta como: \n[[TN, FP], \n[FN, TP]]")
print(conf_matrix_short)


--- Matriz de Confusión  _short---
La matriz se interpreta como: 
[[TN, FP], 
[FN, TP]]
[[4838 1709]
 [1003 9703]]
